In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [5]:
# ==================================================
#  MNIST データセットの読み込み
# ==================================================
(ds_train_raw, ds_test_raw), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# 前処理
def preprocess_train(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, (784,))
    return image, image

batch_size = 256

ds_train = (
    ds_train_raw
    .map(preprocess_train)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

ds_test = (
    ds_test_raw
    .map(preprocess_train)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# ==================================================
# オートエンコーダの定義
# ==================================================
# 入力（784次元）
input_img = keras.Input(shape=(784,), name="input")

# エンコーダ：784 → 256 → 128 → 64 →32
x = layers.Dense(256, activation="relu")(input_img)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)

# 潜在表現
latent = layers.Dense(32, activation="relu", name="latent")(x)

# デコーダ：32 → 64 → 128 → 256 → 784
x = layers.Dense(32, activation="relu")(latent)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(256, activation="relu")(x)
output_img = layers.Dense(784, activation="sigmoid", name="output")(x)

autoencoder = keras.Model(input_img, output_img, name="autoencoder")
autoencoder.summary()

In [ ]:
# ==================================================
# 学習設定
# ==================================================
autoencoder.compile(
    optimizer="adam",
    loss="binary_crossentropy"
)

# ==================================================
# 学習
# ==================================================
history = autoencoder.fit(
    ds_train,
    epochs=30,
    validation_data=ds_test
)

In [ ]:
# ==================================================
# エンコーダ：入力 → 潜在表現
# ==================================================
encoder = keras.Model(input_img, latent, name="encoder")

sample_batch = next(iter(ds_train))[0]
z = encoder(sample_batch)
print("Latent shape:", z.shape)

# ==================================================
# MNIST（ラベル付き）
# ==================================================
def preprocess_with_label(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, (784,))
    return image, label

ds_test_vis = (
    ds_test_raw
    .map(preprocess_with_label)
    .batch(batch_size)
)

In [ ]:
# ==================================================
# 入力画像・再構成画像
# ==================================================

 # 表示枚数
n = 10

x_test, y_test = next(iter(ds_test_vis))
decoded_imgs = autoencoder.predict(x_test)

plt.figure(figsize=(20, 5))

for i in range(n):
    label_num = int(y_test[i].numpy())

    # 入力画像
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(tf.reshape(x_test[i], (28, 28)), cmap="gray")
    plt.title(f"Input\nLabel: {label_num}")
    plt.axis("off")

    # 再構成画像
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(tf.reshape(decoded_imgs[i], (28, 28)), cmap="gray")
    plt.title("Reconstructed")
    plt.axis("off")

plt.show()

# ==================================================
# 学習曲線（損失）
# ==================================================
plt.figure(figsize=(8, 5))
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)
plt.show()
